In [1]:
script = '''
contract store_value(admin: address){

    let storage: nat ;

    entry replace(new_value: nat){
        storage = new_value;
    } 

} '''

In [2]:
from lexer.tzscript_lexer import TzScriptLexer
from lexer.lex_token import Token
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast
# words_separated_by_spaces = ['contract','get_fib_n','(','n',':','int',')','{','let','last_fib_calculated','=', '0',';','entry','get_fib_n','(','n',':','nat',')','{','let','result', '=', 'fib','(','n',')',';','last_fib_calculated', '=', 'result',';','}','func', 'fib','(','n',':', 'nat',')',':', 'nat','{','if', '(','n', '<=', '1',')','{','return', 'n',';','}','else','{','return', 'fib','(','n', '-', '1',')', '+', 'fib','(','n', '-', '2',')',';','}','}','}']
# table = [(TZSCRIPT_GRAMMAR[tok], tok) for tok in words_separated_by_spaces]
lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(script))
tokens = process_lexer_tokens(lexer_tokens)
# expected_tokens = [ Token(x[1], x[0]) for x in table]

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

0 	 <program> -> .contractid(<param-list>){<stat_program_list>}, 
	 S' -> .<program>,  

1 	 <program> -> contract.id(<param-list>){<stat_program_list>},  

2 	 <program> -> contractid.(<param-list>){<stat_program_list>},  

3 	 <program> -> contractid(.<param-list>){<stat_program_list>}, 
	 <param-list> -> .<param>,<param-list>, 
	 <param> -> .id:type, 
	 <param-list> -> .<param>, 
	 <param-list> -> .),  

4 	 <param> -> id.:type,  

5 	 <param> -> id:.type,  

6 	 <param> -> id:type.,  

7 	 <param-list> -> ).,  

8 	 <param-list> -> <param>.,<param-list>, 
	 <param-list> -> <param>.,  

9 	 <param-list> -> <param>,.<param-list>, 
	 <param-list> -> .<param>,<param-list>, 
	 <param> -> .id:type, 
	 <param-list> -> .<param>, 
	 <param-list> -> .),  

10 	 <param-list> -> <param>,<param-list>.,  

11 	 <program> -> contractid(<param-list>.){<stat_program_list>},  

12 	 <program> -> contractid(<param-list>).{<stat_program_list>},  

13 	 <def-entry> -> .entryid(<param-list>){<stat_list>

In [3]:
productions, operations = derivation

operations


['SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE']

In [4]:
productions

[<param> -> id : type,
 <param-list> -> <param>,
 <storage> -> let id : type ;,
 <stat_program> -> <storage>,
 <stat_program_list> -> <stat_program>,
 <param> -> id : type,
 <param-list> -> <param>,
 <atom> -> id,
 <factor> -> <atom>,
 <term> -> <factor>,
 <expr> -> <term>,
 <var-call> -> id = <expr> ;,
 <stat> -> <var-call>,
 <stat_list> -> <stat>,
 <def-entry> -> entry id ( <param-list> ) { <stat_list> },
 <stat_program> -> <def-entry>,
 <stat_program_list> -> <stat_program_list> <stat_program>,
 <program> -> contract id ( <param-list> ) { <stat_program_list> }]

In [5]:
ast = build_slr_ast(productions, operations, tokens)

In [6]:

from visitors.type_check_visitor import TypeCheckVisitor
type_visitor = TypeCheckVisitor()

In [7]:
from visitors.scope_check_visitor import ScopeCheckVisitor
scope_visitor = ScopeCheckVisitor()

In [8]:
# from visitors.string_rep_visitor import StringReprVisitor
# string_visitor = StringReprVisitor(0)

In [9]:
# string_visitor.visit_program(ast)

In [10]:
# print(string_visitor.result)

In [11]:
from visitors.string_rep_visitor import FormatVisitor
format = FormatVisitor()
print(format.visit(ast))

\__ProgramNode: contract node.idx(admin : address) [<stat>; ... <stat>;]
	\__AttrDeclarationNode: admin : address
		\__DeclarationStorageNode: storage : nat
		\__EntryDeclarationNode: Entry replace(new_value : nat)
			\__AttrDeclarationNode: new_value : nat
				\__VarCallNode: storage = <expr>
					\__ VariableNode: new_value
